In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
from Gpt4_OpenAI_Prompt import system_msg,context,postprocess

In [2]:
from Newprompt import system_msg as newprompt

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [5]:
model_name = "NousResearch/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#Testing prompt

# question = "what are the sales in africa"
# prompt = f"""[INST]<<SYS>>
# Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
# For example if user gives "top  orders in march 2020", you should only return march 2020.

# <</SYS>>
# {question}
# [/INST]
# [END]"""

In [7]:
def gereneate_output_llama2(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
    return response

In [12]:
################################################################################
# Setting the same prompt which is used for GPT4 open AI
################################################################################

def get_user_question_for_open_ai_prompt(question):
    #return "Based on the following context:\n" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now return a response for this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt=system_msg):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [END]"""
def post_process_open_ai_prompt_response(output):
    mql = output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    #return mql,reason

In [14]:
%%time
question = "top 5 segments basis discount"
output=gereneate_output_llama2(compose_prompt_open_ai_prompt(question))
post_process_open_ai_prompt_response(output)

**************************************************MQL**************************************************


The response for the question "top 5 segments basis discount" is as follows:

{
"MEASURE": {
"n-gram matched to MEASURE": [
{
"ENTITY": "Matched MEASURE",
"MEASURE CONSTRAINT": [
{
"COMPARISON VALUE": "top 5",
"COMPARISION OPERATOR": "="
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DIMENSION": {
"n-gram matched to DIMENSION": [
{
"ENTITY": "Matched DIMENSION",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"FILTER": {
"n-gram matched to FILTER": [
{
"ENTITY": "Matched FILTER",
"PARENT": "Segment",
"EXCLUDE": ""
}
],
"DERIVED MEASURE": {
"n-gram matched to DERIVED MEASURE": [
{
"ENTITY": "Matched DERIVED MEASURE",
"RATIO FILTER": [
{}
],
"APPLIED MEASURE": [
{
"n-gram matched to MEASURE": [
{
"ENTITY": "Discount",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
]
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DATE VARIABLE": {
"asked time ele

In [16]:
newprompt1 = """You are an assistant for a question answering system that analyzes user questions. Your task is to identify and extract various entities from the user's input and provide a structured response.

**Step 1: Entity Identification**
Your first task is to identify entities in the user's question. These entities include dimensions (e.g., "segments," "products"), measures (e.g., "sales," "discount"), adjectives (e.g., "top," "bottom"), ranks (e.g., "top 5," "bottom 3"), and other relevant elements.

**Step 2: Entity Relationships**
Once you identify these entities, determine any relationships between them. For instance, if the user asks for the "top 5 segments by sales," you should recognize "top" as the adjective, "5" as the rank value, "segments" as the dimension, and "sales" as the measure. Ensure that the relationships between these entities are correctly understood.

**Step 3: Generate a Structured Response**
Based on the entities identified and their relationships, construct a structured response. If adjectives, ranks, or other modifiers are present, include them in the response. The response should be structured into categories for dimensions, measures, and any other relevant details.

**Example:**
- User Input: "Show me the top 5 segments by sales."
- Expected Response: 
  {
      "DIMENSION": {
          "segments": [
              {
                  "ENTITY": "Segment",
                  "RANK": {
                      "RANK ADJECTIVE": "top",
                      "RANK VALUE": "5"
                  }
              }
          ]
      },
      "MEASURE": {
          "sales": [
              {
                  "ENTITY": "Sales"
              }
          ]
      }
  }

**Remember:**
- Be attentive to modifiers, ranks, and relationships between entities.
- Ensure the response is structured and contains relevant information based on the user's input.
- And your response should be in strictly in a Json format (which I have mentioned in the example)

Provide your response.
Also provide the reasoning
"""

In [18]:
newprompt2 = """You are an assistant that helps to map the user question to the context for a question answering system. You might also need to act as a time tagger expert to convert the date elements present in the question to a standard format and to find possible date ranges for the same.

Step 1: Identify the n-grams match between the question and context
Map the n-gram or their lemma or their inflections from the question with the 'other names' in the passed context. Always consider the longest n-gram match, not the sub-string. If there are multiple matches for an n-gram with context, return all such ENTITY in response. If you are returning any match which is not exactly present with the 'other names', make sure that it is a noun phrase and there is a high similarity between the match and the matched "ENTITY".

Step 2: Applying other conditions
Once the match is identified, the next step is to identify other conditions from the user question and apply them to the identified matches. Refer to the following statements to understand different types of conditions to be applied:

1. METRIC CONSTRAINT: METRIC can be MEASURE or DERIVED MEASURE. The user is asking for a comparison limit to be applied to the METRIC. It has two parts: "COMPARISON VALUE" is the value applied to a METRIC, and "COMPARISON OPERATOR" is the operator (in symbols) applied between METRIC and COMPARISON VALUE.

2. ADJECTIVE and TONE: Identify the adjectives (like least, highest performing, etc.) applied to the matched ENTITY. TONE is the intent of the adjective and can be positive or negative.

3. EXCEPTION: Excluded FILTER of a DIMENSION asked in the question, if any. DIMENSION should be the parent of the FILTER. Add a key "EXCLUDE" for such excluded FILTERS and set the value as "True" in the response.

4. RANK: Rank applied to a DIMENSION, if any, like top 5, bottom 3, etc. It has two parts: "RANK ADJECTIVE," the adjective like top, bottom, etc., and "RANK VALUE," a number that comes along with the RANK ADJECTIVE, immediately before or after. If there is no explicit RANK VALUE in the question, make it 1. Based on the meaning of the RANK ADJECTIVE, make it either "top" or "bottom."

5. RATIO FILTERS: This is applicable only for ENTITY "Ratio" in DERIVED MEASURE. Identify the FILTER on which Ratio needs to be calculated.

6. APPLIED MEASURES: This is applicable only for DERIVED MEASURE. Identify the MEASURE on which the DERIVED MEASURE needs to be calculated.

Step 3: Applying time tagger rules (ONLY IF time elements are present in the question)
Identify the TIME ELEMENTS in the input question and convert them to a standard format (if not already) by applying the general time tagging rules. If the TIME ELEMENT is already in a standard format, then no need to convert it. TIME ELEMENT can be either a temporal interval (across months, YoY, MoM, QoQ, WoW, quarterly, etc.) or a temporal expression (time points such as specific dates, relative expressions, etc.). Calculate date ranges for each time point based on specific conditions.

Step 4: Creating the response JSON
Strictly return the response in the exact same JSON format as follows, filling in the information identified from the previous steps. The keys mentioned in upper case in the response are constant.

Example 1 - User input: "show me the 2 top segments basis sales"
- Your response should be:
{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "top",
                        "RANK VALUE": "2"
                    }
                ]
            }
        ]
    },
    "MEASURE": {
        "sales": [
            {
                "ENTITY": "Sales"
            }
        ]
    }
}

Provide reasoning for the response.
"""

In [17]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question,base_prompt=newprompt1))
try:
    post_process_open_ai_prompt_response(output)
except Exception as e:
    print(output)

**************************************************MQL**************************************************


**Step 1: Entity Identification**

* "top" - adjective
* "5" - rank value
* "segments" - dimension
* "discount" - measure

**Step 2: Entity Relationships**

* "top" and "5" are related as the adjective and rank value, respectively.
* "segments" and "discount" are related as the dimension and measure, respectively.

**Step 3: Generate a Structured Response**

{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": {
                    "RANK ADJECTIVE": "top",
                    "RANK VALUE": "5"
                }
            }
        ]
    },
    "MEASURE": {
        "discount": [
            {
                "ENTITY": "Discount"
            }
        ]
    }
}

**************************************************Reason**************************************************
* The user has asked for the top 5 segments based o

In [19]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question,base_prompt=newprompt2))
try:
    post_process_open_ai_prompt_response(output)
except Exception as e:
    print(output)

**************************************************MQL**************************************************


For the question "top 5 segments basis discount," the assistant would follow the same steps as before to identify the relevant entities, measures, and conditions. Here's the response in JSON format:

{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "top",
                        "RANK VALUE": "5"
                    }
                ]
            }
        ]
    },
    "MEASURE": {
        "discount": [
            {
                "ENTITY": "Discount"
            }
        ]
    }
}

**************************************************Reason**************************************************
* Identified n-gram match: "top 5 segments"
* Applied conditions:
	+ METRIC CONSTRAINT: The comparison value is "top" and the operator is " ".
	+ ADJECTIVE: "di

In [20]:
%%time
question = "show me the 2 top segments basis sales"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question,base_prompt=newprompt2))
try:
    post_process_open_ai_prompt_response(output)
except Exception as e:
    print(output)

[INST]<<SYS>>
            You are an assistant that helps to map the user question to the context for a question answering system. You might also need to act as a time tagger expert to convert the date elements present in the question to a standard format and to find possible date ranges for the same.

Step 1: Identify the n-grams match between the question and context
Map the n-gram or their lemma or their inflections from the question with the 'other names' in the passed context. Always consider the longest n-gram match, not the sub-string. If there are multiple matches for an n-gram with context, return all such ENTITY in response. If you are returning any match which is not exactly present with the 'other names', make sure that it is a noun phrase and there is a high similarity between the match and the matched "ENTITY".

Step 2: Applying other conditions
Once the match is identified, the next step is to identify other conditions from the user question and apply them to the identif